In [ ]:
import pandas as pd
import csv
from collections import OrderedDict

# Turn CSV into a pandas DataFrame

raw_data_table = pd.read_csv('./spreadsheets/L1EM_MB_upstream_neurons.csv', sep=',')

# Show first 10 rows of table
#raw_data_table[:10]

In [ ]:
existing_IDs = pd.read_csv('./spreadsheets/existing_ID.csv', sep=',')
ID_lookup = dict(zip(existing_IDs.Keys,existing_IDs.Values))#FBbt IDs for existing terms
#ID_lookup

In [ ]:
#generate FBbt IDs for all new neurons (keys are short names)
start = 49008
new_term_IDs = list()
    
for i in raw_data_table.index:
    x = start + i
    ID = "FBbt:000"+str(x)
    new_term_IDs.append(ID)

new_ID_dict = OrderedDict(zip(raw_data_table.short,new_term_IDs))
ID_lookup.update(new_ID_dict)
#ID_lookup

In [ ]:
#save new short name to FBbt mapping as tsv (needed for MB synapsing template)
with open('./spreadsheets/FBbt_shortname.tsv', 'w') as f:
    for key in new_ID_dict.keys():
        f.write("%s\t%s\n"%(key,new_ID_dict[key]))

In [ ]:
group_text = OrderedDict([('FBN' , "Larval mushroom body one-step feedback neuron"), \
                          ('FAN' , "Larval mushroom body feed across neuron"), \
                          ('FB2N', "Larval mushroom body two-step feedback neuron"), \
                          ('FFN', "Larval mushroom body feedforward neuron")])

In [ ]:
def label_maker(short):
    group_num = short.split("-")
    return (group_text[group_num[0]]).lower() + " " + group_num[1]

In [ ]:
#Short name to long name dict

#MBE neurons (keys are short names)
MBE_table = pd.read_csv('./spreadsheets/MBINs_MBONs.csv')
longname_dict = OrderedDict(zip(MBE_table.short,MBE_table.Name))
longname_dict.update(group_text)
for s in raw_data_table.short:
    longname_dict[s] = label_maker(s)

#longname_dict

In [ ]:
# Make a dictionary with key = column header & value = template specification (first row of table).

#ID column (optionally add type (default 'class') and class_type (default 'subclass') columns here)

template_seed = OrderedDict([('ID' , 'ID')])

#label, description, short synonym:

template_seed.update([("Name" , "A rdfs:label"), ("Definition" , "A IAO:0000115"),\
                      ("Xref_def" , ">A oboInOwl:hasDbXref"),\
                      ("created_by" , "A dc:contributor"),\
                      ("creation_date", "A dc:date")])

#short name synonyms
template_seed.update([("Synonym" , "A oboInOwl:hasExactSynonym"),\
                      ("syn_ref" , ">A oboInOwl:hasDbXref")]) 
                      

# Column for group:
template_seed.update([("Group" , "SC %")])

#Column for NTs
template_seed.update([("Neurotransmitter" , "SC 'capable of' some %")])

#Synapsed_by
template_seed.update([("Synapsed_by" , "SC 'synapsed by' some % SPLIT=|")])
    
#Synapsed_to
template_seed.update([("Synapsed_to" , "SC 'synapsed to' some % SPLIT=|")])
    
# Create dataFrame for template
# from_records takes a list of dicts - one for each row.  We only have one row.

template = pd.DataFrame.from_records([template_seed])

template

In [ ]:
nt_names = OrderedDict([("Glu" , "glutamatergic"), ("GABA" , "GABAergic"), ("ACh" , "cholinergic")])

In [ ]:
def name_lister(names):
    L = ""
    if len(names) < 1:
        return False
    elif len(names) == 1:
        return names[0]
    elif len(names) > 1:
        L = names[0]
        if len(names) > 2:
            for i in names[1:-1]:
                L = L + ", " + i
        L = L + " and " + names[-1]
        return L

In [ ]:
count = 0 #0 = FBN-1

for i in raw_data_table.index:

    r = raw_data_table.short[count]
    row_od = OrderedDict([]) #new template row as an empty ordered dictionary
    for c in template.columns: #make columns and blank data for new template row
        row_od.update([(c , "")])
    
    #these are the same in each row
    row_od["Xref_def"] = "doi:10.1101/649731"
    row_od["syn_ref"] = "doi:10.1101/649731"
    row_od["created_by"] = "http://orcid.org/0000-0002-1373-1705"
    row_od["creation_date"] = "2020-03-05T12:00:00Z"

    #easy to generate data
    row_od["ID"] = new_ID_dict[r]
    row_od["Synonym"] = r
    row_od["Name"] = label_maker(r)

    #Group
    row_od["Group"] = ID_lookup[r.split("-")[0]]
    
    #neurotransmitter
    if pd.notna(raw_data_table.NT[count]):
        row_od["Neurotransmitter"] = ID_lookup[raw_data_table.NT[count]]
    else: row_od["Neurotransmitter"] = ""
    
    #synapsing
    if pd.notna(raw_data_table.synapsed_by[count]):
        synby_ID_list = [ID_lookup[x] for x in raw_data_table.synapsed_by[count].split(" ")]
        synby_ID_string = ""
        for i in synby_ID_list:
            synby_ID_string = synby_ID_string + i + " | "
        row_od["Synapsed_by"] = synby_ID_string.rstrip(" |")
        
    if pd.notna(raw_data_table.synapsed_to[count]):
        synto_ID_list = [ID_lookup[x] for x in raw_data_table.synapsed_to[count].split(" ")]
        synto_ID_string = ""
        for i in synto_ID_list:
            synto_ID_string = synto_ID_string + i + " | "
        row_od["Synapsed_to"] = synto_ID_string.rstrip(" |")    
    
    #####STUFF FOR DEFINITION
    #group
    r_group = raw_data_table.Group[count]
    group_def = group_text[r_group]

    #neurotransmitter
    if pd.notna(raw_data_table.NT[count]):
        nt_text = " and they are %s (Eschbach et al., 2019)."%(nt_names[raw_data_table.NT[count]])
    else: nt_text = "."
            
    #synapsing
    if pd.notna(raw_data_table.synapsed_by[count]):
        synby_text_list = raw_data_table.synapsed_by[count].split(" ")
        synby_text_list2 = [i + "s" if i in ["MBIN, ""MBON", "FBN", "FAN", "FB2N", "FFN"] \
                            else i for i in synby_text_list]
        synby_text = " that receives synaptic input from %s"\
        %(name_lister(synby_text_list2))
    else: synby_text = ""

    if len(synby_text) > 0:
        if pd.notna(raw_data_table.synapsed_to[count]):
            synto_text_list = raw_data_table.synapsed_to[count].split(" ")
            synto_text_list2 = [i + "s" if i in ["MBIN, ""MBON", "FBN", "FAN", "FB2N", "FFN"] \
                            else i for i in synto_text_list]
            synto_text = " and outputs onto %s (Eschbach et al., 2019)."\
            %(name_lister(synto_text_list2))
        else: synto_text = " (Eschbach et al., 2019)."
    elif pd.notna(raw_data_table.synapsed_to[count]):
        synto_text_list = raw_data_table.synapsed_to[count].split(" ")
        synto_text_list2 = [i + "s" if i in ["MBIN, ""MBON", "FBN", "FAN", "FB2N", "FFN"] \
                            else i for i in synto_text_list]
        synto_text = " that outputs onto %s (Eschbach et al., 2019)."\
        %(name_lister(synto_text_list2))
    else: synto_text = " (Eschbach et al., 2019)."
        
    row_od["Definition"] = group_def + synby_text + synto_text + \
    " There is one bilateral pair of these cells" + nt_text
            
    
    #make new row into a DataFrame and add it to template
    new_row = pd.DataFrame.from_records([row_od])
    template = pd.concat([template, new_row], ignore_index=True, sort=False)

    count +=1
    
template

In [ ]:
template.to_csv("./template.tsv", sep = "\t", header=True, index=False)